# Position Based Dynamics

We will follow along with this presentation of PBD: https://matthias-research.github.io/pages/publications/posBasedDyn.pdf

The basic equations for position based dynamics converted to Geometric Algebra are very similar.  

The gradient operator in GA using index notation is $e_i \partial_i$

To formulate a Constraint Projection solver we start with a constraint equation, $C(p)$, that take positions as arguments and will equal zero when it is satisfied.

Taylor expansion of a scalar valued function using the differential

$C(p + \Delta p) \approx C(p) + \Delta p_i \partial_i C(p) = 0$ &emsp; &emsp; (1)

We want the gradient operator to appear.  We use the fact that, for our euclidean basis,

$e_i \cdot e_j = \delta_{ij}$ &emsp; &emsp; (2) 

Note: if we were using a non-euclidean metric, this would need to be modified.

$C(p) + \Delta p_i e_i \cdot e_i \partial_i C(p) = C(p) + \Delta p \cdot \nabla C(p) = 0$ &emsp; &emsp; (3)

Ok, we've recovered the first Position Based Dynamics formula, while setting ourselves up for extension to non-eucledian metrics.

$\Delta p = \lambda \nabla C(p)$   &emsp; &emsp; (4)
    
Solving for $\Delta p$ in terms of known values gives

$\Delta p = \frac{-C(p)}{\nabla C(p) \cdot \nabla C(p)} \nabla C(p)$ &emsp; &emsp; (5)


## G(n,0,0) the algebra of directions 

The most familiar algebra for Position Based Dynamics will be G(n,0,0). This means all the basis vectors square to 1.  This is the metric of euclidean space.

The gradient operator is slightly different.  Rather than organizing the resulting partial derivatives in a column vector, partials are attached to basis vectors.

$\nabla f(p) = e_1 \partial_x f(p) + e_2 \partial_y f(p) + e_3 \partial_z f(p)$

rather than

$\nabla f(p) = \begin{bmatrix}
\partial_x f(p) \\ 
\partial_y f(p)\\ 
\partial_z f(p)
\end{bmatrix}$

we will use (1,2,3) rather than (x,y,z) to specify partial derivatives from now on.

If you wish you can treat the gradient operator as a 1-vector in GA just as you can in Linear Algebra.

## G(n,0,1) the algebra of planes.  aka PGA

Our first non-euclidean metric.  This algebra has n basis vectors that square to 1 and a single basis vector that squares to 0.  We will assume n = 3 for now.
The basis vector that squares to zero is similar to the w component of homogeneous vectors you may be familiar with.
Effectively this lets us efficiently model projective geometry.  
We choose $e_{n}$ as our degenerate basis vector.  Since Julia is our implementation language we stick with 1-based indexing and choose to place the degenerate basis element as the last element of a 1-vector to mirror the use of w in a Vector Algebra setting.  i.e. $e_1*e_1 == 1$, $e_2*e_2 == 1$, $e_3*e_3 == 1$, and $e_4*e_4 == 0$ for 3D projective geometry.

One more thing we need to do for maximum elegance and power is to represent our points as 3-vectors, which is visualized as the intersection of 3 planes.  This can take a little while to get used to.  For more information about PGA please check out https://bivector.net

This unreasonably effective algebra has many advantages.  Almost trivial representation/construction of Dual Quaternions and the ability to represent objects at infinity being it's two main super-powers.

### PBD using PGA

A few things we need to consider now that we have a degenerate metric and trivector based positions.

We need to revisit our foundational equations and see what happens with these changes.

The degenerate metric prevents us from using the previous trick we used to get a gradient operator (2), as it's not true for $e_4$.

Instead we multiply (2) by the pseudoscalar $\textit{I} = e_{1234}$.  
We can factor the pseudoscalar into the 1-vectors for the gradient and the 3-vectors for points.  Factorization is via the wedge product.

$e_{1234} C(p) + -1^{(4-i)} p_i e_{jkl} \wedge e_i \partial_i = e_{1234} C(p) + -1^{(4-i)} \Delta p \wedge \nabla C(p)$ &emsp; &emsp; (6)

Note that $\partial_4 C(p) = 0$ since the coordinate function for the projective plane is constant for the objects we are interested in.  This leaves us with an ideal point, which is a direction vector, as we would expect.

which gives us a gradient operator for PGA that looks like this

$\nabla_* = -e_1 \partial_1 + e_2 \partial_2 - e_3 \partial_3$

if we had a different ordering of basis indices we'd potentially have different signs here.  As we will see later, this doesn't matter since the signs will cancel.

$C(p + \Delta p) \approx \textit{I} C(p) + \Delta p \wedge \nabla_* C(p) = 0$
 
The dualized statement of D'Alembert's principle (4) becomes

$\Delta p = \lambda e_{ij4} \partial_k C(p)$

Which says that any virtual displacement consistent with the constraint must be aligned with it's (dual) gradient.

Some algebra where we use the identity $(\textit{I}A \wedge B) = \textit{I}(A \cdot B)$ and $e_ke_k = 1, k \in {1,2,3}$

$-\textit{I} C(p) = \lambda e_{ij4} e_k e_k \partial_k C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} \nabla_* C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} (\nabla_* C(p) \cdot \nabla_* C(p))$

$-C(p) = \lambda \nabla_* C(p) \cdot \nabla_* C(p)$

Solving for $\Delta p$ as before and noticing $\nabla_* \cdot \nabla_*$ will cancel any sign differences from $\nabla$

$\Delta p = \frac{-C(p)}{\nabla C(p)  \cdot \nabla C(p)} e_{ij4} \partial_k C(p)$

There we have it.  The Constraint Projection Equations of PBD for PGA.  The only difference is that we have ideal points ( directions ) represented as trivectors instead of 1-vectors as seen in the standard formulation.  This is what we would expect. 

In PGA we need to translate objects with a translational dual quaternion.

# Rigid Body Dynamics


## Useful identities

### Jacobian of commutator for bivectors

The matrix form of the Linear Transform representing the cross product operator $\mathbf{x}\times$ is

$\begin{bmatrix}
0 & -z & y \\ 
z & 0  & -x \\ 
-y & x & 0
\end{bmatrix}$

choose coordinate functions $x,y,z$ for a bivector
such that $B(\mathbf{x}) = x(e_2 \wedge e_3) + y(e_3 \wedge e_1) + z(e_1 \wedge e_2)$ 

The equivalent for the commutator product $B(\mathbf{x}) \times$ applied to bivectors is

$\begin{bmatrix}
0 & z & -y \\ 
-z & 0  & x \\ 
y & -x & 0
\end{bmatrix}$

It's the same matrix but negated.






In [3]:
# Optional: 
# These commands tell the Julia package manager to use the exact
# set of dependencies specified in the Project.toml file in this folder. 
# That should give you a nice, reproducible environment for testing. 

using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(Pkg.PackageSpec(path=dirname(@__DIR__)))
Pkg.instantiate()

 Activating environment at `~/lab/GBD.jl/notebooks/Project.toml`
Path `/Users/mewert/lab/GBD.jl` exists and looks like the correct package. Using existing path.
  Resolving package versions...
No Changes to `~/lab/GBD.jl/notebooks/Project.toml`
No Changes to `~/lab/GBD.jl/notebooks/Manifest.toml`


In [4]:
using StaticArrays, LinearAlgebra, ReferenceFrameRotations

In [25]:
# Good old linear algebra based RBD solver ( aside from quaternions )

Vector3d = SVector{3,Float64}
Matrix3d = SMatrix{3,3,Float64}

mutable struct RigidBodyVA
    x::SVector{3,Float64}
    R::SMatrix{3,3,Float64} # body to world transform wTb
    m::Float64              # move to end for better cache perf
    I::SVector{3,Float64}   # bodyspace
    v::SVector{3,Float64}
    w::SVector{3,Float64}   # world space
    q::Quaternion{Float64}
end

function setrotation!(rb, R::Matrix3d)
    rb.R = R
    rb.q = dcm_to_quat(R')
end

function setrotation!(rb, q::Quaternion{Float64})
    rb.q = q
    rb.R = quat_to_dcm(q)'
end

getrotation(rb) = rb.R

RigidBodyVA(x,m=1.0,inertia=ones(3)) = RigidBodyVA(x, Matrix{Float64}(I, 3,3), m, inertia, 
                                                   zeros(3), zeros(3), Quaternion(1.0,0,0,0))


skew(w) = [0.0 -w[3] w[2];
         w[3] 0.0  -w[1];
         -w[2] w[1] 0.0]

function inertia_tensor(box::SVector{3,Float64}, m::Float64)
    x,y,z = box
    (m/12.0) * Vector3d(y^2+z^2, x^2+z^2, x^2+y^2)
end


function wIw_implicit(ω₀::Vector3d, It, wTb::Matrix3d, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    ω = wTb'*ω₀
    Iω = It*ω
    f = ω×Iω*dt 
    df = It + (skew(ω)*It - skew(Iω))*dt
   
    ω′ = ω - df\f
    wTb*ω′
end

function integrate!(rb::RigidBodyVA, dt::Float64)
    rb.x += rb.v*dt
    wIw = wIw_implicit(rb.w, Diagonal(rb.I), getrotation(rb), dt)
    rb.w = wIw
    q2 = rb.q + 0.5*dt*rb.w*rb.q
    setrotation!(rb, q2/norm(q2))
    
    return rb
end

function integrate_explicit!(rb::RigidBodyVA, dt::Float64)
    rb.x += rb.v*dt
    # torque free Euler's equation
    # Iω̇ = -ω×(Iω)
    wIw = -cross(rb.w, rb.R*Diagonal(rb.I)*rb.R'*rb.w)
    rb.w += rb.R*inv(Diagonal(rb.I))*rb.R'*wIw*dt
    q2 = rb.q + 0.5*dt*rb.w*rb.q
    setrotation!(rb, q2/norm(q2))
    
    return rb
end


integrate_explicit! (generic function with 1 method)

In [6]:
using Multivectors

Base.:~(k::K) where {K<:KVector} = reverse(k)
Base.:~(k::K) where {K<:Multivector} = reverse(k)

In [7]:
# G(3) based RBD solver.  as in "Geometric Algebra for Physicists.  Doran, Lasenby"

module G3
    using Multivectors
    @generate_basis("+++")
end

G3Vector = KVector{Float64,1}
G3Bivector = KVector{Float64,2}
G3Rotor = Multivector{Float64,2}

g3𝐼 = 1.0G3.e₁₂₃
g3vec(x,y,z) = Float64(x)*G3.e₁ + Float64(y)*G3.e₂ + Float64(z)*G3.e₃ 
g3bivec(x,y,z) = Float64(x)*G3.e₂₃ - Float64(y)*G3.e₁₃ + Float64(z)*G3.e₁₂

bivec(M::MT) where MT<:Multivector = grade(M,2)


mutable struct RigidBodyG3
    x::G3Vector
    R::G3Rotor # body to world transform wTb
    m::Float64  # move to end for better cache perf
    I::G3Bivector   # bodyspace
    v::G3Vector
    w::G3Bivector   # world space
end

function inertia_tensor(box::G3Vector, m)
    x,y,z = coords(box)
    (m/12.0) * ((y^2+z^2)*G3.e₂₃ - (x^2+z^2)*G3.e₁₃ + (x^2+y^2)*G3.e₁₂)
end

#!me be very explicit about it for now.
apply_inertiai(It, B, eij, ei) = (It⋅eij)*(B⋅eij)*g3𝐼*ei 
apply_inertia(It::G3Bivector, B::G3Bivector) = apply_inertiai(It, B,  1.0G3.e₁₂, 1.0G3.e₃) +
                                               apply_inertiai(It, B, -1.0G3.e₁₃, 1.0G3.e₂) +
                                               apply_inertiai(It, B,  1.0G3.e₂₃, 1.0G3.e₁)
apply_inertia_invi(It, B, eij, ei) = (1.0/(It⋅eij))*(B⋅eij)*g3𝐼*ei 
apply_inertia_inv(It::G3Bivector, B::G3Bivector) = apply_inertia_invi(It, B,  1.0G3.e₁₂, 1.0G3.e₃) +
                                                   apply_inertia_invi(It, B, -1.0G3.e₁₃, 1.0G3.e₂) +
                                                   apply_inertia_invi(It, B,  1.0G3.e₂₃, 1.0G3.e₁)
#!me coord approach more efficient, but need to watch out for sign flips
#apply_inertia(It::G3Bivector, B::G3Bivector) = (coords(B) .* coords(It)) .* [1.0G3.e₁₂, -1.0G3.e₁₃, 1.0G3.e₂₃]

RigidBodyG3(x,m=1.0,inertia=inertia_tensor(g3vec(1,1,1), 1)) = RigidBodyG3(x, 1.0+g3bivec(0,0,0), 
                                                                             m, inertia, 
                                                                             g3vec(0,0,0), 
                                                                             g3bivec(0,0,0))
bivec_coords(B) = [B⋅~(1.0G3.e₂₃), B⋅(1.0G3.e₁₃), B⋅~(1.0G3.e₁₂)]

function ∇wIw(ω, Iw)
    Iwx,Iwy,Iwz = bivec_coords(Iw)
    x,y,z = bivec_coords(ω) 
    x*(-Iwz-Iwy+2.0Iwx)*1.0G3.e₁+ y*(-Iwz-Iwx+2.0Iwy)*1.0G3.e₂+ z*(-Iwy-Iwx+2.0Iwz)*1.0G3.e₃
end

# this solves L(ω₀)+L̇(ω′)*dt=0 for ω′ 
# f(ω) = I(ω₀) - I⁻¹(ω′×I(ω′))*dt = 0
# f(ω) ≈ f(ω₀ + dω*h) ≈ f(ω₀) + ∇f(ω₀)*(ω-ω₀)
# mult by I
# f(ω) = ω₀ - ω′×I(ω′)*dt = 0
# f(ω) ≈ f(ω₀ + dω*h) ≈ f(ω₀) + ∇f(ω₀)*(ω-ω₀) = 0
# ∇f(ω₀)*ω₀ - f(ω₀) = ∇f(ω₀)*ω′
# ω′ = (∇f(ω₀)*ω₀ - f(ω₀))/∇f(ω₀)
# ω
# 
# returns ω̇
function wIw_newton(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    ω = bivec(~R*ω₀*R)
    Iw = apply_inertia(It, ω)
    
    # dt cancels 
    #!me or squares?  check work
    f = bivec(-ω×Iw)
    ∇f = ∇wIw(ω, Iw)
   
    # the dual(∇wIw) has same coords as -∇wIw. which cancels - from eqn we are solving
    f_∇f = bivec(dual(f)/∇f)/dt   # vector/vector = bivector
    
    bivec(R*apply_inertia_inv(It, f_∇f)*~R)
end

# this solves for ω
function wIw_implicit(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    ω = bivec(~R*ω₀*R)
    Iw = apply_inertia(It, ω)
    f = bivec(-ω×Iw*dt)
    df = It + bivec(-ω×It*dt)
   
    ω′ = ω - df/f
    # this is a multivector...
    R*ω′*~R
end

function wIw_explicit(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    ω = bivec(~R*ω₀*R)
    Iw = apply_inertia(It, ω)
    dω = bivec(ω×Iw*dt)
    bivec(R*apply_inertia_inv(It, dω)*~R)
end

function integrate!(g3rb::RigidBodyG3, dt::Float64)
    g3rb.x += g3rb.v*dt
    
    wIw = wIw_explicit(g3rb.w, g3rb.I, g3rb.R, dt)   #bivec(g3rb.w × (g3rb.R*g3rb.I*g3rb.w*~g3rb.R))
    #wIw = wIw_newton(g3rb.w, g3rb.I, g3rb.R, dt)   #bivec(g3rb.w × (g3rb.R*g3rb.I*g3rb.w*~g3rb.R))
    g3rb.w += wIw*dt
    #g3rb.w += apply_inertia_inv(g3rb.I, wIw*dt)
    
    #wIw = wIw_implicit(g3rb.w, g3rb.I, g3rb.R, dt)
    #g3rb.w = wIw
    g3rb.R = normalize(g3rb.R - 0.5*g3rb.w*g3rb.R*dt)
    
    return g3rb
end

function wIw_implicit_matrix(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    #ω = wTb'*ω₀
    ω = bivec(~R*ω₀*R)
    #Iω = It*ω
    Iω = apply_inertia(It, ω)
    #f = ω×Iω*dt
    f = bivec(-ω×Iω*dt)
    It3x3 = Diagonal(bivec_coords(It))
    
    #!me this gives correct behavior, but is not very GA
    df = It3x3 + (skew(coords(dual(ω)))*It3x3 - skew(coords(dual(Iω))))*dt
    
    # try with jacobian of df but stay in GA.  here ω is a bivector  ( i.e. ω = Ω from GA 4 Physicists)
    # f(x + h) ≈ f(x) + f′ₓ(h)
    #
    # note ′ now means differential.  use ₀ and ₁ to mean current and next ω or f
    # ω₁ = ω₀ + dω
    # dω = ω₁ - ω₀
    #
    # f(ω + dω) ≈ f(ω) + f′_ω(dω);  one step taylor 
    #
    # use formula for (external-)torque free change in angular momentum
    # L̇ = f(ω) = I(dω) - ω×I(ω) = 0 
    # I(ω₁) = I(ω₀) - ω₁×I(ω₁) = f(ω₁)
    #
    # get newton's method with taylor expansion
    #
    # f(ω₁) = f(ω₀ + dω) =  I(ω₁) ≈ I(ω₀) - ω₀×I(ω₀) + (f′₀(ω₁))
    # = I(ω₀) - ω₀×I(ω₀) + Jω₁
    
    # Jacobian of B×
    
    
    
    
    
    #!me incorrect behavior.   maybe the derivative of -ω×Iω is wrong?
    #df = It3x3 + skew(coords(dual(bivec(-ω×It*dt))))
   
    # dual of dual in for bivector in R3 yields extra -sign
    ω′ = -dual(KVector(coords(dual(ω)) - df\coords(dual(f))))
    bivec(R*ω′*~R)
end

function integrate_matrix!(g3rb::RigidBodyG3, dt::Float64)
    g3rb.x += g3rb.v*dt
    
    #!me booooo! going to a matrix
    wIw = wIw_implicit_matrix(g3rb.w, g3rb.I, g3rb.R, dt)
    g3rb.w = wIw
    g3rb.R = normalize(g3rb.R - 0.5*g3rb.w*g3rb.R*dt)
    
    return g3rb
end


integrate_matrix! (generic function with 1 method)

In [8]:
using MeshCat, GeometryBasics, CoordinateTransformations

┌ Info: Precompiling MeshCat [283c5d60-a78f-5afe-a0af-af636b173e11]
└ @ Base loading.jl:1278


In [9]:
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/mewert/.julia/packages/MeshCat/rJuuK/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [10]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [11]:
box_radius = Vec(0.05,0.2,0.4)
offset2 = Vec(0,1.0,0)

setobject!(vis[:box1], Rect(-box_radius, box_radius*2))
setobject!(vis[:box2], Rect(-box_radius, box_radius*2))

MeshCat Visualizer with path /meshcat/box2 at http://127.0.0.1:8700

In [30]:
anim = Animation()

function torot(R::M) where M<:Multivector
    k,j,i = coords(grade(R,2))
    quat_to_dcm(Quaternion(grade(R,0), -i, j, -k))'
end

rb = RigidBodyVA(zeros(3))
rb.x = [0,-0.5,0]
setrotation!(rb, Matrix3d(I))
#[0.866  0  0.5
       # 0      1  0
       # -0.5      0  0.866]
rb.w = [0.0,10,1]
rb.v = [0.0,-0.1,0.0]
rb.m = 10
rb.I = inertia_tensor(Vector3d(2*box_radius), rb.m)

g3rb = RigidBodyG3(g3vec(0,0.5,0))
g3rb.v = g3vec(0,0.1,0)
g3rb.w = g3bivec(0,10,1)
g3rb.m = 10
g3rb.I = inertia_tensor(g3vec((2*box_radius)...), g3rb.m)

atframe(anim, 0) do
    # within the context of atframe, calls to 
    # `settransform!` and `setprop!` are intercepted
    # and recorded in `anim` instead of having any
    # effect on `vis`.
    settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(rb.R)))
    settransform!(vis[:box2], compose(Translation(coords(g3rb.x)...), LinearMap(torot(g3rb.R))))
end

dt = 0.0033

for i in 1:10*24*2
    atframe(anim, i) do
        integrate_explicit!(rb, dt)
        integrate_matrix!(g3rb, dt)
        settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(rb.R)))
        settransform!(vis[:box2], compose(Translation(coords(g3rb.x)...), LinearMap(torot(g3rb.R))))
    end
end

# `setanimation!()` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
setanimation!(vis, anim)

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul

┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.julia/packages/Multivectors/Yp50z/src/KVectors.jl:77
┌ Warning: resolving Blade types in top-level module via dual(1)
└ @ Multivectors /Users/mewert/.jul